In [1]:
import pandas as pd
import datetime
import numpy as np

# Template preprocessing

Read in the template that has the burn activity of the different fires over time, and get the fire IDs represented.

In [2]:
template = pd.read_csv('temporalBurnArea.csv')
fires = template.FireID.unique()
print(fires)
output_data = []

[13 21 22 26 24 27 31 28]


The template needs some work. It was generated in Libreoffice by selecting only certain rows from a CSV file which had emissions for each fire ID. However, not every hour contained data for every fire. We first need to ensure that every fire is represented once and only once for each hour of the simulation. When finished, the template should contain one record per fire per timestep. First, we need to get the start and end times. 

In [3]:
def get_bounds() : 
    first_rec = template.loc[0] 
    start = datetime.datetime(int(first_rec.Year), 
                              int(first_rec.Month), 
                              int(first_rec.Day),
                              int(first_rec.Hour), 
                              int(first_rec.Minute), 0)

    last_rec = template.loc[template.shape[0]-1]
    last = datetime.datetime(int(last_rec.Year), 
                             int(last_rec.Month), 
                             int(last_rec.Day),
                             int(last_rec.Hour), 
                             int(last_rec.Minute), 0)
    
    delta = last - start
    delta_hours = int((delta.days * 24) + (delta.seconds / 3600))
    return start, last, delta_hours

start, last, delta_hours = get_bounds()
delta_hours


72

In [4]:
def lookup_from_template(index) : 
    """Returns the first entry matching the simulation time and fire ID.
       If there is no such entry, it returns None."""
    sim_time = index[0]
    fire     = index[1]
    # lookup record in template
    matching = ((template.Year == sim_time.year) &
                (template.Month == sim_time.month) &
                (template.Day == sim_time.day) & 
                (template.Hour == sim_time.hour) & 
                (template.FireID == fire))
    
    retval = None
    if np.count_nonzero(matching) > 0 : 
        first_row = np.nonzero(matching.values)[0][0]
        retval = template.iloc[[first_row]]
        
    return retval

def lookup_fireid(the_fire) : 
    """Finds and returns the first record pertaining to the given fire id."""
    matching = template.FireID == the_fire
    first_row = np.nonzero(matching.values)[0][0]
    return template.loc[[first_row]]
    

# construct a comprehensive set of indices for the output
indices = [ ( (start + (i*datetime.timedelta(hours=1))), f) for i in range(delta_hours) for f in fires ]
        
        
        

Here is where we do a couple of things. First, we create a new template from the data in the CSV file (read in previously.) We do this a little laboriously because there are some instances where there are duplicate emissions for the same fire in the same time period. By using a loop, we can make sure there's only one value. A loop also allows us to find all those fire+time combinations which have no emissions data in the CSV file. Once known, we can pad out the template with dummy values for each timestep where they are missing. We do not want to shut the fire completely off, but are turning it down to a minimum value. 

Note that when we turn the fire off, we set it's emission to 2 "units per hour". In the particle dump output, you can identify these "off" particles as having a mass of 2/60/(*number of processors*).

In [5]:

full_template=None
cntr=0
for i in indices : 
    thisval = lookup_from_template(i)
    if thisval is None : 
        firstfire = lookup_fireid(i[1])
        thisval = pd.DataFrame({"Year":int(i[0].year), "Month":int(i[0].month), "Day":int(i[0].day),
                             "Hour":int(i[0].hour), "Minute":0, "Duration":100, 
                             "Latitude":firstfire.Latitude, "Longitude":firstfire.Longitude, "Height":10, 
                             "fcon_Mg":0, 
                             "Area":0, 
                             "Power":0, 
                             "FireID":i[1]})

    thisval.index = [cntr]
    if full_template is not None : 
        full_template = full_template.append(thisval)
    else :
        full_template = thisval
        
    cntr = cntr+1

min_emission_val=2
full_template.loc[full_template.fcon_Mg == 0, 'fcon_Mg'] = min_emission_val

# Sparsifying the matrix

Now expand the template out, yielding one record for each pollutant per timestep per fireID. HYSPLIT's mechanism for this allows you to specify any amount of pollutant release for each source location. To do this, you repeat the _entire_ list of source locations for each pollutant. We are only going to allow a single fire to release a single type of pollutant. Hence we are going to walk through the list of fire IDs, and turn the fire down to a set minimum value. This is a different set minimum value than above, as this represents the value of a pollutant emitted by a fire which should not be emitting that tracer at all.

Note we have chosen to set this "should not be emitting that tracer at all" value to 1 mass unit per hour. You can identify these particles in the particle dump by looking for a mass of 1./60/(*number of processors*).

In [6]:
for fire in fires : 
    d = full_template.copy()
    d.loc[d['FireID'] != fire, 'fcon_Mg'] = 1
    output_data.append(d)

In [7]:
output_df = pd.concat(output_data, ignore_index=True, sort=False)
del output_df['FireID']

In [8]:
output_df['fcon_Mg']

0         2
1         1
2         1
3         1
4         1
       ... 
4603      1
4604      1
4605      1
4606      1
4607    692
Name: fcon_Mg, Length: 4608, dtype: int64

In [9]:
output_df[output_df['fcon_Mg']!=0]

,Year,Month,Day,Hour,Minute,Duration,Latitude,Longitude,Height,fcon_Mg,Area,Power
0,2017,8,16,5,0,100,47.06566,-114.8391,10,2,0,0
1,2017,8,16,5,0,100,46.04124,-113.5958,10,1,0,0
2,2017,8,16,5,0,100,46.42476,-113.6029,10,1,0,0
3,2017,8,16,5,0,100,47.09658,-113.7502,10,1,0,0
4,2017,8,16,5,0,100,46.69792,-114.2481,10,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4603,2017,8,19,4,0,100,47.09558,-113.7485,10,1,0,0
4604,2017,8,19,4,0,100,46.70335,-114.2284,10,1,0,0
4605,2017,8,19,4,0,100,47.23767,-113.4225,10,1,0,0
4606,2017,8,19,4,0,100,45.41659,-115.1784,10,1,0,0


In [10]:
template[template['fcon_Mg']!=0]

,Year,Month,Day,Hour,Minute,Duration,Latitude,Longitude,Height,fcon_Mg,Area,Power,FireID
2,2017,8,16,5,0,100,46.42476,-113.6029,10,933,0,0,22
3,2017,8,16,5,0,100,47.09658,-113.7502,10,67,0,0,26
6,2017,8,16,6,0,100,46.42476,-113.6029,10,1119,0,0,22
7,2017,8,16,6,0,100,47.09658,-113.7502,10,166,0,0,26
10,2017,8,16,7,0,100,46.42476,-113.6029,10,839,0,0,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,2017,8,19,3,0,100,47.02962,-112.7147,10,259,0,0,28
438,2017,8,19,4,0,100,47.23767,-113.4225,10,1854,0,0,27
439,2017,8,19,4,0,100,47.02962,-112.7147,10,692,0,0,28
440,2017,8,19,5,0,100,47.06641,-114.8431,10,961,0,0,13


In [11]:
output_df.to_csv('multiple_output', header=False, index=False, sep=' ', float_format='%8.5f')

In [12]:
just_locations = pd.DataFrame([full_template['Latitude'], full_template['Longitude'], full_template['Height']]).T
just_locations.to_csv('fire_locations',header=False, index=False, sep=' ', float_format='%8.5f')

In [13]:
just_locations

,Latitude,Longitude,Height
0,47.06566,-114.8391,10.0
1,46.04124,-113.5958,10.0
2,46.42476,-113.6029,10.0
3,47.09658,-113.7502,10.0
4,46.69792,-114.2481,10.0
...,...,...,...
571,47.09558,-113.7485,10.0
572,46.70335,-114.2284,10.0
573,47.23767,-113.4225,10.0
574,45.41659,-115.1784,10.0
